# Data Science Methods for Clean Energy Research 
## _Neural Networks_

## Outline

### 1. Load dataset & rescale
### 2. Build a NN model with Keras
### 3. Train NN and test
### 4. Increase complexity of NN

In [ ]:
# Import plotting libraries
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.size': 22})

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set larger fontsize for all plots
%load_ext autoreload
%autoreload 2


In [ ]:
import tqdm
from tqdm.keras import TqdmCallback

### 1. Loading the data, splitting into train and test and rescaling

Set our random seed so that all computations are deterministic

In [ ]:
seed = 42

Read in the raw data for the HCEPDB into a pandas dataframe

In [ ]:
df = pd.read_csv('datasets/HCEPDB_moldata.csv')
df.head()

In [ ]:
df_smaller = df.sample(frac=0.05)

In [ ]:
df_smaller.describe()

Separate out the input features from the output target `'pce'`

In [ ]:
X = df_smaller[['mass', 'voc', 'jsc', 'e_homo_alpha', 'e_gap_alpha', 
        'e_lumo_alpha']].values
Y = df_smaller[['pce']].values

Let's create the test / train split from this data and keep 20% for testing.  

In [ ]:
X_train_pn, X_test_pn, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.20,
                                                    random_state=seed)

Now we need rescale the input features of the training set using the `StandardScaler()` class [more info](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and store the scaler to use it for the future test sets. We are rescaling input features because we do not want one of the input features to matter more than the others.

In [ ]:
# create the scaler from the training data only and keep it for later use
X_train_scaler = StandardScaler().fit(X_train_pn)
# apply the scaler transform to the training data
X_train = X_train_scaler.transform(X_train_pn)

Now let's reuse that scaler transform on the test set.  This way we never contaminate the test data with the training data.  We'll start with a histogram of the testing data just to prove to ourselves it is working.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].hist(X_test_pn[:,0], alpha=0.6, color='mediumvioletred')
ax[0].set_xlabel('mass')
ax[0].set_ylabel('count')
ax[0].set_title('mass distribution',fontsize=18)
ax[1].hist(X_test_pn[:,1], alpha=0.6, color='royalblue')
ax[1].set_xlabel('voc')
ax[1].set_ylabel('count')
ax[1].set_title('voc distribution',fontsize=18)
plt.tight_layout()
plt.show()

OK, bnow apply the training scaler transform to the test and plot a histogram

In [ ]:
X_test = X_train_scaler.transform(X_test_pn)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].hist(X_test[:,0], alpha=0.6, color='mediumvioletred')
ax[0].set_xlabel('mass')
ax[0].set_ylabel('count')
ax[0].set_title('mass distribution',fontsize=18)
ax[1].hist(X_test[:,1], alpha=0.6, color='royalblue')
ax[1].set_xlabel('voc')
ax[1].set_ylabel('count')
ax[1].set_title('voc distribution',fontsize=18)
plt.tight_layout()
plt.show()

### 2. Creating the neural network model

This is a simple neural network with no hidden layers and just the inputs transitioned to the output. Here we will use `Keras` functions [Keras documentation](https://keras.io/guides/). We will use `Dense` layers as defined [here](https://keras.io/api/layers/core_layers/dense/), and the `Adam` [optimizer](https://keras.io/api/optimizers/adam/) which relies on gradient descent.

In [ ]:
def simple_network():
    # assemble the structure
    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
my_model = simple_network()
my_model.summary()

### 3. Training the NN

Train the neural network with the following

In [ ]:
# initialize the random number generator - i.e. get reproducible starting weights
np.random.seed(seed)
# create the NN framework
estimator = KerasRegressor(build_fn=simple_network,
        epochs=150, batch_size=10000, verbose=0)
history = estimator.fit(X_train, y_train, validation_split=0.33, epochs=150, 
        batch_size=10000, verbose=0, callbacks=TqdmCallback(verbose=0))

The history object returned by the `fit` call contains the information in a fitting run.

In [ ]:
print(history.history.keys())

In [ ]:
print("final MSE for train is %.2f and for validation is %.2f" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

Let's plot it!

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'], '--', c='b')
plt.plot(history.history['val_loss'], c='crimson')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Let's get the MSE for the test set.

In [ ]:
test_loss = estimator.model.evaluate(X_test, y_test)
print("test set mse is %.2f" % test_loss)

So our train mse is **very similar** to the training and validation at the final step!

###  Let's look at another way to evaluate the set of models using cross validation

Use 10 fold cross validation to evaluate the models generated from our training set.  We'll use scikit-learn's tools for this.  Remember, this is only assessing our training set.  If you get negative values, to make `cross_val_score` behave as expected, we have to flip the signs on the results (incompatibility with keras).

In [ ]:
kfold = KFold(n_splits=10, random_state=seed, shuffle=True)

results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (-1 * results.mean(), results.std()))

### 4. Increase complexity of NN

Let's use a hidden layer this time. Note: it is worthwhile to test how the final loss changes with number of epochs of training as well as learning rate. 

In [ ]:
def medium_network():
    # assemble the structure
    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    opt = keras.optimizers.Adam(learning_rate=0.9)
    # compile the model
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [ ]:
# initialize the andom seed as this is used to generate
# the starting weights
np.random.seed(seed)
# create the NN framework
estimator = KerasRegressor(build_fn=medium_network,
        epochs=150, batch_size=10000, verbose=0)
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0, callbacks=TqdmCallback(verbose=0))
print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'],'--',c='b')
plt.plot(history.history['val_loss'],c='crimson')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim([6,8])
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
test_loss = estimator.model.evaluate(X_test, y_test)
print("test set mse is %.3e" % test_loss)

In [ ]:
def medium_network(lr=0.8):
    # assemble the structure
    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    opt = keras.optimizers.Adam(learning_rate=lr)
    # compile the model
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [ ]:
# initialize the andom seed as this is used to generate
# the starting weights
np.random.seed(seed)
# create the NN framework
MSE = []
lr_vals = [0.00001, 0.0001, 0.01, 0.1, 1.0]
for l in lr_vals:
    estimator = KerasRegressor(build_fn=medium_network, 
                               epochs=150, batch_size=10000, 
                               verbose=0, lr=l)
    history = estimator.fit(X_train, y_train, validation_split=0.30,
                            epochs=150, batch_size=10000, 
                            verbose=0, callbacks=TqdmCallback(verbose=0))
    print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))
    MSE.append([history.history['loss'][-1], 
                history.history['val_loss'][-1],
                estimator.model.evaluate(X_test, y_test)])
    
    

In [ ]:
plt.plot(lr_vals, [m[0] for m in MSE],'-o',lw=3,label='train')
plt.plot(lr_vals, [m[1] for m in MSE],'-.s',lw=3, label='validation')
plt.plot(lr_vals, [m[2] for m in MSE],':>',lw=3, label='test')  
plt.xlabel('learning rate')
plt.ylabel('loss')
plt.legend()
plt.show()